# RUNTIME CONTEXT

LangChain's `create_agent` runs on LangGraphs runtime under the hood

Langchain exposes a Runtime object with following information
- **CONTEXT**: statis information like userid, db connections or other dependencies for an agent invocation
- **Store**: a`BaseStore` instance used for long-term memory
- **Stream Writer**: an object used for streaming informaion via the `"custom"` Stream mode

*RUNTIME INFORMATION CAN BE ACCESSED IN TOOLS AS WELL AS VIA CUSTOIM AGENT MIDDLEWARE*


## ACCESS RUNTIME CONTEXT 
- specify a `context_schema` to define the structure of the `context` stored in the agent runtime
- Use the `get_runtime` function to access the Runtime object inside a tool
- When invoking the agent, pass the `context` argument with relevant configurartion for the run
- 

In [1]:
import utility as ut

OPENAI_API_KEY = ut.OPENAI_API_KEY
LANGSMITH_API_KEY = ut.LANGSMITH_API_KEY
MODEL_NAME = ut.MODEL_NAME

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio
import asyncio

nest_asyncio.apply()

mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": [ "-y", "@theo.foobar/mcp-time"],
        }
    },
)


mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP Tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP Tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


In [3]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model = MODEL_NAME,
    tools = mcp_tools,
    system_prompt = "you are a helpful assistant",
)

In [4]:
result = await agent_with_mcp.ainvoke(
    {
        "messages": [
            {
                "role":"user",
                "content":"what is the time in London right now?"
            }
        ]
    }
)

for msg in result["messages"]:
    msg.pretty_print()

/Users/mia/myenv/lib/python3.11/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


================================ Human Message =================================

what is the time in London right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (call_5CnNNfPulhJwD5rdTxrCtBRz)
 Call ID: call_5CnNNfPulhJwD5rdTxrCtBRz
  Args:
    timezone: Europe/London
================================= Tool Message =================================
Name: current_time

2025-11-20T20:34:20Z
================================== Ai Message ==================================

The current time in London is 8:34 PM on November 20, 2025.


In [5]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [6]:

from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [7]:
from langchain_core.tools import tool
from langgraph.runtime import  get_runtime

@tool
def execute_sql(query:str) -> str:
    """Execute a SQL Lite command and return results"""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [8]:
SYSTEM_PROMPT = """ You are a careful SQLite Analyst

Rules:
 - Think Step-by-Step.
 - When you need data, call the tool `execute_sql` with one SELECT query.
 - Read-Only only; No INSERT/UPDATE/DELETE/ALTER/DROP/TRUNCATE/CREATE/REPLACE.
 - Limit to 5 rows of output unless the user explicitely asks otherwise.
 - if the tool returns 'Error:', revise the SQL and try again
 - Prefer explicit column lists; avoid SELECT *
 - If you are getting wrong answer you can always list the tables in the db and check within those tables 
"""

In [9]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
agent = create_agent(
    model= MODEL_NAME ,
    tools=[execute_sql],
    system_prompt = SYSTEM_PROMPT,
    context_schema = RuntimeContext,
)

In [10]:
question = "This is Frank Harris, what was the total on my last invoice"
steps=[]
for step in agent.stream(
    {"messages": question},
    context=RuntimeContext(db=db),
    stream_mode="values",
): 
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, what was the total on my last invoice
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_dcfRE6l98M8td2DzYg5hOV5J)
 Call ID: call_dcfRE6l98M8td2DzYg5hOV5J
  Args:
    query: SELECT amount FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT amount FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_RX9MEBHrvglFRG1c1tLig2d2)
 Call ID: call_RX9MEBHrvglFRG1c1tLig2d2
  Args:
    query: SELECT name FROM sqlite_master WHE

In [11]:
question = "What were the titles?"

for step in agent.stream(
    {
        "messages": [
            {
                "role":"user",
                "content": question
            }
        ]
    },
    context=RuntimeContext(db=db),
    stream_mode="values",
): 
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_7j1ye0JUlAiUwvh5DhSTvXaO)
 Call ID: call_7j1ye0JUlAiUwvh5DhSTvXaO
  Args:
    query: SELECT title FROM movies LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: movies
[SQL: SELECT title FROM movies LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_DE9230jiuQtU6awA0HStQwBO)
 Call ID: call_DE9230jiuQtU6awA0HStQwBO
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), (

# ADD MEMORY

In [12]:
from langgraph.checkpoint.memory import InMemorySaver

In [13]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
agent = create_agent(
    model= MODEL_NAME ,
    tools=[execute_sql],
    system_prompt = SYSTEM_PROMPT,
    context_schema = RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [14]:
question = "This is Frank Harris, what was the total on my last invoice"
steps=[]
for step in agent.stream(
    {
        "messages": [
            {
                "role":"user",
                "content": question
            }
        ]
    },
    {
        "configurable": {
            "thread_id": "1"
        }
    },
    context=RuntimeContext(db=db),
    stream_mode="values",
): 
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, what was the total on my last invoice
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_3PNM2n45C8iwBGT4tDPIJK8N)
 Call ID: call_3PNM2n45C8iwBGT4tDPIJK8N
  Args:
    query: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_bA1fpTKtZsEZJnoMIUKTy94y)
 Call ID: call_bA1fpTKtZsEZJnoMIUKTy94y
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
=

In [15]:
question = "What were the titles?"
steps = []
for step in agent.stream(
    {
        "messages": [
            {
                "role":"user",
                "content": question
            }
        ]
    },
    {
        "configurable": {
            "thread_id": "1"
        }
    },
    context=RuntimeContext(db=db),
    stream_mode="values",
): 
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_z3w6c5Hua1TlOaf8lJ4iahQh)
 Call ID: call_z3w6c5Hua1TlOaf8lJ4iahQh
  Args:
    query: SELECT Track.Name FROM InvoiceLine JOIN Track ON InvoiceLine.TrackId = Track.TrackId WHERE InvoiceLine.InvoiceId = (SELECT InvoiceId FROM Invoice WHERE CustomerId = (SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris') ORDER BY InvoiceDate DESC LIMIT 1);
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',), ('So What',)]
================================== Ai Message ==================================

The titles on your last invoice were:

1. Holier Than Thou
2. Through The Never
3. My Friend Of Misery
4. The W